<a href="https://www.kaggle.com/code/zheyunshou/titanic?scriptVersionId=109449434" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()
#合并训练数据与测试数据
full=train_data.append(test_data,ignore_index=True)
print(full.shape)
full.info()

(1309, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [3]:
#数据类型缺失
#Age
full['Age']=full['Age'].fillna(full['Age'].mean())#用平均值填充
#Fare
full['Fare']=full['Fare'].fillna(full['Fare'].mean())
#字符串类型缺失
#Embarked，缺失2
full['Embarked']=full['Embarked'].fillna('S')#用出现最频繁的值填充
#Cabin，缺失值较多
full['Cabin']=full['Cabin'].fillna('U')#U表示Unknown

In [4]:
full.describe()#确认填充

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,12.883193,1.041658,0.865560,51.738879
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,22.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,29.881138,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,35.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [5]:
#特征提取-性别
full['Sex'].head()
sex_mapDict={'male':1,'female':0}
sex=pd.DataFrame()
sex['Sex']=full['Sex'].map(sex_mapDict)
#sex.head()

#特征提取-登船港口
full['Embarked'].head()
embarkedDf=pd.DataFrame()
embarkedDf=pd.get_dummies(full['Embarked'],prefix='Embarked')
#embarkedDf.head()

#特征提取-客舱等级
full['Pclass'].head()
PclassDf=pd.DataFrame()
PclassDf=pd.get_dummies(full['Pclass'],prefix='Pclass')
#PclassDf.head()

#特征提取-姓名（头衔、称谓）
def getTitle(Name):
    str1=Name.split(',')[1]
    str2=str1.split('.')[0]#Mr
    str3=str2.strip()
    return str3

titleDf1=full['Name'].map(getTitle)
#titleDf1.head()
#存放提取后的特征
titleDf=pd.DataFrame()
titleDf['Title']=full['Name'].map(getTitle)
#titleDf.head()

titleDf=pd.get_dummies(titleDf['Title'])
titleDf.head()
#头衔字符串与头衔类别映射
titleDf=pd.DataFrame()
titleDf['Title']=full['Name'].map(getTitle)
title_mapDict={
    'Capt':'Officer',
    'Col':'Officer',
    'Major':'Officer',
    'Jonkheer':'Royalty',
    'Don':'Royalty',
    'Sir':'Royalty',
    'Dr':'Officer',
    'Rev':'Officer',
    'the Countess':'Royalty',
    'Dona':'Royalty',
    'Mme':'Mrs',
    'Mlle':'Miss',
    'Ms':'Mrs',
    'Mr':'Mr',
    'Mrs':'Mrs',
    'Miss':'Miss',
    'Master':'Master',
    'Lady':'Royalty'
}

titleDf['Title']=titleDf['Title'].map(title_mapDict)
titleDf=pd.get_dummies(titleDf['Title'])
#titleDf.head()

#特征提取-家庭类别
familyDf=pd.DataFrame()
familyDf['FamilySize']=full['Parch']+full['SibSp']+1
familyDf[1:10]
#分成大中小家庭
familyDf['Family_Single']=familyDf['FamilySize'].map(lambda s:1 if s==1 else 0)
familyDf['Family_Small']=familyDf['FamilySize'].map(lambda s:1 if 2<=s<=4 else 0)
familyDf['Family_Large']=familyDf['FamilySize'].map(lambda s:1 if 5<=s else 0)
#familyDf.head()

#特征提取-客舱号
cabinDf=pd.DataFrame()
full['Cabin']=full['Cabin'].map(lambda c:c[0])
cabinDf=pd.get_dummies(full['Cabin'],prefix='Cabin')
cabinDf.head()


,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U
0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1


In [6]:
#特征选择
#相关性矩阵
corrDf=full.corr()
corrDf
#联系之前提取的有用的数据特征
full_1=pd.concat([full['Age'],
                  full['PassengerId'],
                  titleDf,
                  PclassDf,
                  familyDf,
                  full['Fare'],
                  cabinDf,
                  embarkedDf,
                  sex,
                  full['Survived']],
                 axis=1)
full_1.head()
#各种类别的相关性矩阵
corrDf1=full_1.corr()
corrDf1
#提取相关系数矩阵中的“Survived”，并按降序排列
corrDf1['Survived'].sort_values(ascending=False)
#选择有代表性的特征
full_x1=pd.concat([titleDf,
                  PclassDf,
                  familyDf,
                  full['Fare'],
                  cabinDf,
                  embarkedDf,
                  sex,
                  full['PassengerId']],
                 axis=1)
full_x1.head()

,Master,Miss,Mr,Mrs,Officer,Royalty,Pclass_1,Pclass_2,Pclass_3,FamilySize,...,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S,Sex,PassengerId
0,0,0,1,0,0,0,0,0,1,2,...,0,0,0,0,1,0,0,1,1,1
1,0,0,0,1,0,0,1,0,0,2,...,0,0,0,0,0,1,0,0,0,2
2,0,1,0,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,1,0,3
3,0,0,0,1,0,0,1,0,0,2,...,0,0,0,0,0,0,0,1,0,4
4,0,0,1,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,1,1,5


In [7]:
#构建模型
#拆分原始数据集和预测集
sourceRow=891
source_x=full_x1.loc[0:sourceRow-1,:]#特征
source_y=full.loc[0:sourceRow-1,'Survived']#标签

pred_x=full_x1.loc[sourceRow:,:]

from sklearn.model_selection import train_test_split
#建立模型用的训练集和测试集
train_x,test_x,train_y,test_y=train_test_split(source_x,source_y,train_size=0.8)
#输出数据大小
print('原始数据集特征：',source_x.shape,
     '训练数据集特征：',train_x.shape,
     '测试数据集特征：',test_x.shape)
print('原始数据集标签：',source_y.shape,
     '训练数据集标签：',train_y.shape,
     '测试数据集标签：',test_y.shape)
#建立模型
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(train_x, train_y)
model.score(test_x,test_y)

原始数据集特征： (891, 28) 训练数据集特征： (712, 28) 测试数据集特征： (179, 28)
原始数据集标签： (891,) 训练数据集标签： (712,) 测试数据集标签： (179,)


0.8212290502793296

In [8]:
pred_y= model.predict(pred_x)
pred_y= pred_y.astype(int)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': pred_y})
output.to_csv('submission.csv', index=False)
print("Submission was successfully saved!")

Submission was successfully saved!
